In [3]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

In [4]:
df = pd.read_csv('C:/Users/ACER/Desktop/Data Science/ICT/dataset/labor-neg.data.csv',header = None)
df.columns=['duration','wage1','wage2','wage3','cola','hrs','pension','standby_pay','shift_differential','edu_allowance','holidays','vacation','disability_support',
                         'dental_plan','bereavement','health_plan','class_distribution']
df=df.replace('?', '')

# I will drop standby_pay attribute because of its few data entries, which do not sufficiently provide any useful input for discovery
df=df.drop(['standby_pay'],axis = 1)

### Duration = 1 means that the employee receives only wage1. Duration = 2 means that the employee receives wage1, wage2. Duration = 3 means employees receives wage1,2,3

In [5]:
df.wage2 = df.wage2.replace('',0)
df.wage3 = df.wage3.replace('',0)


In [6]:
for i in range(0,len(df.index)):
    if(df.wage3[i] == '' and df.duration[i] == '1' or df.duration[i] == '2'):
        df.wage3[i] == '0'
    elif (df.wage3[i] == '' and df.duration[i] == '3'):
        df.wage3[i] == df.wage3.mean()       


### Employees who work for more than 38 hours/week will be less likely to receive education allowance because they will not have time to study. Therefore, we will fill education allowance missing values for employees who work for more than 38 hours/week as no, vice versa.
        

In [7]:
df.hrs = pd.to_numeric(df.hrs)
hrs_mean = int(df.hrs.mean())

In [8]:
df.hrs = pd.to_numeric(df.hrs)
df.hrs = df.hrs.replace(np.nan, hrs_mean)
for i in range(0, len(df.index)):
    if(df.loc[i,'edu_allowance'] == '' and df.hrs[i] >= 38):
        df.loc[i, 'edu_allowance'] = 'yes'
        df.edu_allowance[i] = "yes"
    elif (df.loc[i,'edu_allowance'] == '' and df.hrs[i] < 38):
        df.loc[i,'edu_allowance'] = 'no'


# df.edu_allowance = df.edu_allowance.replace('',df.edu_allowance.mode().iloc[0])    
# for i in range(0,len(df.index)):
#     if(df.edu_allowance[i] == 'no' and df.hrs[i] == ''):
#             df.loc[i,'hrs'] = 38
#     elif (df.edu_allowance[i] == 'yes' and df.hrs[i] == ''):
#            df.loc[i,'hrs'] = df.int(pf.to_numeric(df.hrs)).mean()




C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### We notice that if there is bereavement leave, there is a high chance that the company will also provide disability support because I believe that having disability support indicates that the company sympathizes and emphasizes the importance of mental health, hence, offers bereavement leave

In [9]:
df.bereavement = df.bereavement.replace('', (df[df['bereavement']!='']).bereavement.mode().iloc[0])

for i in range(0,len(df.index)):
   if df.loc[i,'disability_support'] == '':
      df.disability_support[i] = df.bereavement[i]
        
for i in range(0, len(df.index)):
    if df.loc[i,'dental_plan'] == '' and df.loc[i,'health_plan']!= '':
        df.dental_plan[i] = df.health_plan[i]
    elif df.loc[i, 'health_plan']=='' and df.loc[i,'dental_plan']!='':
        df.health_plan[i] = df.dental_plan[i]

df.dental_plan = df.dental_plan.replace('', (df[df['dental_plan']!='']).dental_plan.mode().iloc[0])
for i in range(0,len(df.index)):
   if df.loc[i,'disability_support'] == '':
      df.disability_support[i] = df.bereavement[i]
        
for i in range(0, len(df.index)):
    if df.loc[i,'dental_plan'] == '' and df.loc[i,'health_plan']!= '':
        df.dental_plan[i] = df.health_plan[i]
    elif df.loc[i, 'health_plan']=='' and df.loc[i,'dental_plan']!='':
        df.health_plan[i] = df.dental_plan[i]


C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

### We will use mode and mean to fill values in for vacation, shift differential, and holidays respectively. 

In [10]:
type(df['hrs'].mode().iloc[0])
df[df['vacation']!=''].vacation.mode().iloc[0]


'below average'

In [11]:
vacation_mode = df[df['vacation']!=''].vacation.mode().iloc[0]
df.vacation = df.vacation.replace('',vacation_mode)

df.shift_differential = pd.to_numeric(df.shift_differential) 
df.holidays = pd.to_numeric(df.holidays)
mean_shift = (df[df.shift_differential != np.nan]).shift_differential.mean()
df.shift_differential = df.shift_differential.fillna(int(mean_shift))
df.holidays = df.holidays.fillna(int(df.holidays.mean()))



### I see that there is a relationsip between pension and vacation. There exists a pattern where if vacation is generous, then pension is empl_contr. If vacation is average , then pension is ret_allw. And if vacation is below average, then pension is none. A company provides employees pension means the company cares about the welfare, physical and mental health of the employees. Hence, I conclude this relationship. 



In [12]:
for i in range(0, len(df.index)):
    if df.loc[i, 'pension'] == '' and df.loc[i,'vacation'] == 'generous':
        df.loc[i,'pension'] = 'empl_contr'
    elif df.loc[i,'pension'] == '' and df.loc[i,'vacation'] == 'average':
        df.loc[i,'pension'] = 'ret_allw'
    elif df.loc[i, 'pension'] == '' and df.loc[i, 'vacation'] == 'below average':
        df.loc[i, 'pension'] = 'none'


### In the remaining two attributes, I do not see a specific relationship between them and other attributes. Hence, I will be using Naive Bayes to pre-process the data for cola, which is categorical variable.

In [13]:
cols1 = list(df.columns)
cols1.remove('duration')
cols1.remove('pension')
cols1.remove('hrs')
cols1.remove('class_distribution')

df2 = df[df.cola != '']
df.drop(df.index[2])
df2 = df2[cols1]

train_gau = df2[['wage1','wage2','wage3','shift_differential','holidays']].values

train_multi = pd.get_dummies(df2[['vacation','dental_plan','health_plan']]).values

train_be = df2[['edu_allowance','disability_support','bereavement']]
train_be[train_be == "yes"] = 1
train_be[train_be == "no"] = 0
train_be = train_be.values
lb = preprocessing.LabelBinarizer()
lb.fit(['yes','no'])

label = df2[['cola']].values



clf1 = GaussianNB()
clf2 = BernoulliNB(fit_prior=False)
clf3 = MultinomialNB(fit_prior=False)

clf1.fit(train_gau,label)
clf2.fit(train_be,label)
clf3.fit(train_multi,label)

def predict(gau,ber,multi):
    x1 = clf1.predict_proba(gau)
    x2 = clf2.predict_proba(ber)
    x3 = clf3.predict_proba(multi)
    result = x1[0] * x2[0] * x3[0]
    probmax = max(result)
    if probmax == result[0]:
        return clf1.classes_[0]
    if probmax == result[1]:
        return clf1.classes_[1]
    return clf1.classes_[2]

for i in range(0,len(df.index)):
    try:
        if df.cola[i] == '':
            gau = (df[['wage1','wage2','wage3','shift_differential','holidays']].values)[i]     
            
            ber = lb.transform((df[['edu_allowance','disability_support','bereavement']].values)[i])
            ber = np.array([ber[0][0],ber[1,0],ber[2,0]])
            
            multi = pd.get_dummies(df[list(['vacation','dental_plan','health_plan'])])
            columns = pd.get_dummies(df2[list(['vacation','dental_plan','health_plan'])]).columns
            multi = multi[columns].values
            
            df.cola[i] = predict(gau,ber,multi)
    except Exception as ex:
        continue
  
df = df.drop(df.index[2])
df


C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ACER\Anaconda3\lib\site-packages\pandas\core\frame.py:2454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)
C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

,duration,wage1,wage2,wage3,cola,hrs,pension,shift_differential,edu_allowance,holidays,vacation,disability_support,dental_plan,bereavement,health_plan,class_distribution
0,1,5,0,0,none,40.0,ret_allw,2.0,yes,11.0,average,yes,full,yes,full,good
1,2,4.5,5.8,0,none,35.0,ret_allw,4.0,yes,11.0,below average,yes,full,yes,full,good
3,3,3.7,4,5,tc,38.0,none,4.0,yes,11.0,below average,yes,full,yes,full,good
4,3,4.5,4.5,5,tcf,40.0,ret_allw,4.0,yes,12.0,average,yes,half,yes,half,good
5,2,2,2.5,0,none,35.0,ret_allw,6.0,yes,12.0,average,yes,full,yes,full,good
6,3,4,5,5,tc,38.0,empl_contr,4.0,yes,12.0,generous,yes,none,yes,half,good
7,3,6.9,4.8,2.3,none,40.0,none,3.0,yes,12.0,below average,yes,full,yes,full,good
8,2,3,7,0,none,38.0,none,25.0,yes,11.0,below average,yes,half,yes,half,good
9,1,5.7,0,0,none,40.0,empl_contr,4.0,yes,11.0,generous,yes,full,yes,full,good
10,3,3.5,4,4.6,none,36.0,empl_contr,3.0,no,13.0,generous,yes,full,yes,full,good


In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
le = preprocessing.LabelEncoder()

In [15]:
#Using dummies to encode 2-value attributes:
dummies_data = pd.get_dummies(df[['bereavement','disability_support']])
categorical_data = df[['vacation','dental_plan','health_plan','pension']]
continuous_data = df[['wage1', 'wage2', 'wage3','shift_differential','holidays']]
_class_distribution = df.class_distribution

#Using Label Encoder to give numerical values to values to attributes:
categorical_data = categorical_data.apply(le.fit_transform)
train = pd.concat([dummies_data, categorical_data,continuous_data], axis=1)
train.head(57)



,bereavement_no,bereavement_yes,disability_support_no,disability_support_yes,vacation,dental_plan,health_plan,pension,wage1,wage2,wage3,shift_differential,holidays
0,0,1,0,1,0,0,0,2,5,0,0,2.0,11.0
1,0,1,0,1,1,0,0,2,4.5,5.8,0,4.0,11.0
3,0,1,0,1,1,0,0,1,3.7,4,5,4.0,11.0
4,0,1,0,1,0,1,1,2,4.5,4.5,5,4.0,12.0
5,0,1,0,1,0,0,0,2,2,2.5,0,6.0,12.0
6,0,1,0,1,2,2,1,0,4,5,5,4.0,12.0
7,0,1,0,1,1,0,0,1,6.9,4.8,2.3,3.0,12.0
8,0,1,0,1,1,1,1,1,3,7,0,25.0,11.0
9,0,1,0,1,2,0,0,0,5.7,0,0,4.0,11.0
10,0,1,0,1,2,0,0,0,3.5,4,4.6,3.0,13.0


In [16]:
param_grid = [(10,5),(8,7),(8,2),(10,4)]
for i in range(0,len(param_grid)):
    outer_cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=i)
    mclf = MLPClassifier(solver='lbfgs', random_state=i,hidden_layer_sizes = param_grid[i])
    print (cross_val_score(mclf, X=train, y=_class_distribution, cv=outer_cv).mean())

0.8125
0.848958333333
0.645833333333
0.859375


### Conclusion: the highest testing accuracy in this case is 10 nodes for input layers and 4 nodes for hidden layers.